In [ ]:
#|default_exp frontmatter

# frontmatter

> A YAML and formatted-markdown frontmatter processor
- order: 9

In [ ]:
#|export
from nbdev.imports import *
from nbdev.process import *
from nbdev.doclinks import _nbpath2html

from execnb.nbio import *
from fastcore.imports import *
import yaml

In [ ]:
#|hide
from fastcore.test import *

In [ ]:
#|hide
_test_file = '../../tests/docs_test.ipynb'

In [ ]:
#|export
_RE_FM_BASE=r'''^---\s*
(.*?\S+.*?)
---\s*'''

_re_fm_nb = re.compile(_RE_FM_BASE+'$', flags=re.DOTALL)
_re_fm_md = re.compile(_RE_FM_BASE, flags=re.DOTALL)

_re_fm_title_desc = re.compile(r'^#\s+(\S.*?)(?:\n|$)(?:\s*\n)*(?:>\s+(\S.*?)(?:\n|$)(?:\s*\n)*)?', flags=re.MULTILINE)
_re_fm_kv = re.compile(r'^((?:\s*-\s+[a-zA-Z_][a-zA-Z0-9_]*\s*:\s+.*(?:\n|$)|\s*\n)*)', flags=re.MULTILINE)

def _parse_kv_block(block_text):
    """Parse a block of key-value pairs from lines starting with '-'"""
    if not block_text.strip(): return {}
    kv_lines = []
    for line in block_text.split('\n'):
        line = line.strip()
        if line.startswith('-'):
            kv_content = line[1:].strip()  # Remove the '-' and strip whitespace
            if kv_content: kv_lines.append(kv_content)
    if not kv_lines: return {}
    try: return yaml.safe_load('\n'.join(kv_lines))
    except Exception as e:
        warn(f'Failed to create YAML dict for:\n{kv_lines}\n\n{e}\n')
        return {}

def _md2dict(s:str):
    "Convert H1 formatted markdown cell to frontmatter dict"
    if '#' not in s: return {}
    res = {}
    remaining_start = 0
    
    # Extract title and description
    title_desc_match = _re_fm_title_desc.match(s)
    if not title_desc_match: return {}
    res['title'] = title_desc_match.group(1)
    if title_desc_match.group(2): res['description'] = title_desc_match.group(2)
    remaining_start = title_desc_match.end()
    
    # Extract KV pairs starting from where title/desc ended
    kv_text = s[remaining_start:]
    kv_match = _re_fm_kv.match(kv_text)
    if kv_match:
        kv_dict = _parse_kv_block(kv_match.group(1))
        res.update(kv_dict)
        remaining_start += kv_match.end()
    
    # Extract remaining content
    remaining = s[remaining_start:].strip()
    if remaining: res['__remaining'] = remaining
    
    return res

def _fm2dict(s:str, nb=True):
    "Load YAML frontmatter into a `dict`"
    re_fm = _re_fm_nb if nb else _re_fm_md
    match = re_fm.search(s.strip())
    return yaml.safe_load(match.group(1)) if match else {}

In [ ]:
#|export
def _dict2fm(d): return f'---\n{yaml.dump(d)}\n---\n\n'
def _insertfm(nb, fm): nb.cells.insert(0, mk_cell(_dict2fm(fm), 'raw'))

class FrontmatterProc(Processor):
    "A YAML and formatted-markdown frontmatter processor"
    def begin(self): 
        self.fm = getattr(self.nb, 'frontmatter_', {})
        self.is_qmd = hasattr(self.nb, 'path_') and Path(self.nb.path_).suffix == '.qmd'
        
    def _update(self, f, cell):
        s = cell.get('source')
        if not s: return
        d = f(s)
        if not d: return
        remaining = d.pop('__remaining', None)
        self.fm.update(d)
        if remaining:
            new_cell = mk_cell(remaining, 'markdown')
            cell_idx = self.nb.cells.index(cell)
            self.nb.cells.insert(cell_idx + 1, new_cell)
        cell.source = None

    def cell(self, cell):
        if cell.cell_type=='raw': self._update(_fm2dict, cell)
        elif (cell.cell_type=='markdown' and 'title' not in self.fm): self._update(_md2dict, cell)

    def end(self):
        self.nb.frontmatter_ = self.fm
        if not self.fm: return
        if not hasattr(self.nb, 'path_'):
            raise AttributeError('Notebook missing `path_` attribute.\n\nPlease remove any nbdev-related notebook filters '
                                 'from your _quarto.yml file (e.g. `ipynb-filter: [nbdev_filter]`), since they are no '
                                 'longer supported as of nbdev v2.3. See the v2.3 launch post for more information: '
                                 'https://forums.fast.ai/t/upcoming-changes-in-v2-3-edit-now-released/98905.')
        self.fm.update({'output-file': _nbpath2html(Path(self.nb.path_)).name})
        _insertfm(self.nb, self.fm)

YAML frontmatter can be added to notebooks in one of two ways:

1. By adding a raw notebook cell with `---` as the first and last lines, and YAML between them, or
2. A specially formatted markdown cell. The first line should be start with a single `#` (creating an H1 heading), and becomes the title. Then, optionally, a line beginning with `>` (creating a quote block), which becomes the description. Finally, zero or more lines beginning with `- ` (creating a list), each of which contains YAML key-value pairs. (If you already have "title" defined in frontmatter in a raw cell, then markdown cells will be ignored.)

For instance, our test notebook contains the following markdown cell:

```
# a title
> A description
- key1: value1
- key2: value2
- categories: [c1, c2]
```

It also contains the following raw cell:

```
---
execute:
  echo: false
---
```

When we process with `FrontmatterProc`, these will both be removed, and a single raw cell will be added to the top, containing the combined YAML frontmatter:

We similarly need a `write_qmd` function to write a notebook to a .qmd file.

In [ ]:
nbp = NBProcessor(_test_file, procs=FrontmatterProc)
nbp.process()
print(nbp.nb.cells[0].source)

REMAINING With some remaining text that I want to preserve!!

E.g., see this list:

- [With a link](https://example.com)

And this table:

| Column 1 | Column 2 |
|----------|----------|

and this example code:

```python
a = 4 + 6
```

And this image:

![An image](https://example.com/image.png)
---
categories:
- c1
- c2
description: A description
execute:
  echo: false
key1: value1
key2: value2
output-file: docs_test.html
title: a title

---




In addition, a `frontmatter_` attr will be added to the notebook, containing this information as a `dict`:

In [ ]:
d = nbp.nb.frontmatter_
d

{'execute': {'echo': False},
 'title': 'a title',
 'description': 'A description',
 'key1': 'value1',
 'key2': 'value2',
 'categories': ['c1', 'c2'],
 'output-file': 'docs_test.html'}

In [ ]:
#|hide
test_eq(d['description'], 'A description')
test_eq(d['categories'], ['c1','c2'])
test_eq(d['output-file'], 'docs_test.html')

## Export -

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()